In [10]:
import pandas as pd
import re
from collections import defaultdict

In [23]:
# --- Example input: recipes dictionary ---
# Each recipe is a dict of ingredient: quantity (string or number with units)
recipes = {
    "Pork Nectarine Stir Fry": {
        # "Vegetable Oil": "2 tbsp",
        # "Garlic": "3 cloves",
        "Red Chili": "1 pcs",
        "Pork Fillet": "1 lbs",
        'Spinach': '1 pcs',
        'Kaffir Lime Leaves': '3 pcs',
        # 'Soy Sauce': '2.5 tbsp',
        # 'Lime Juice': '2 tsp',
        'Nectarines': '2 pcs',
        # 'White Rice': '2 lbs',
    },
    'Trail Mix Granola Bars': {
        'Honey': '1 cups',
        'Almond Butter': '.5 cups',
        # 'Coconut Oil': '2 tbsp',
        # 'Vanilla Extract': '1 tsp',
        'Pecans': '.75 cups',
        'Cashews': '.5 cups',
        'Dates': '5 pcs',
        'Shredded Coconut': '.25 cups',
        'Raisins': '.25 cups',
        # 'Cranberries': '2 tbsp',
        'Dark Chocolate': '.5 cups',
    },
    'Chinese Beef Stir-Fry': {
        'Sirloin': '.75 lbs',
        # 'Vegetable Oil': '3 tbsp',
        # 'Garlic': '4 cloves',
        'Ginger': '1 tbsp',
        'Red Chili': '2 pcs',
        'Chinese Broccoli': '1 pcs',
        'Snow Peas': '.5 lbs',
        'Bean Sprouts': '.25 lbs',
        'Oyster Sauce': '1 tbsp',
        'Sambal Oelek': '1 tsp',
        # 'White Rice': '2 lbs',
    }
}

# Supported units
SUPPORTED_UNITS = [
    "g", "kg",
    "ml", "l",
    "tbsp", "tsp", 'lbs',
    "pcs", "cloves", "loaf",
    'cups',
]

# Function to parse quantity
def parse_quantity(q):
    # Match number + unit, allowing decimal numbers
    match = re.match(r"^\s*([\d\.]+)\s*([a-zA-Z]+)\s*$", q)
    if match:
        value = float(match.group(1))
        unit = match.group(2).lower()
        if unit in SUPPORTED_UNITS:
            return value, unit
    return None, q  # Return None for value if parsing fails

# Aggregate shopping list
shopping_list = defaultdict(lambda: defaultdict(float))
notes = defaultdict(list)

for ingredients in recipes.values():
    for ingredient, quantity in ingredients.items():
        value, unit_or_note = parse_quantity(quantity)
        if value is not None:
            shopping_list[ingredient][unit_or_note] += value
        else:
            notes[ingredient].append(unit_or_note)

# Build final table
rows = []
for ingredient, units_dict in shopping_list.items():
    combined_qtys = [f"{val:g} {unit}" for unit, val in units_dict.items()]
    note_str = ", ".join(notes[ingredient]) if notes[ingredient] else ""
    rows.append({
        "Ingredient": ingredient,
        "Total Quantity": ", ".join(combined_qtys),
        "Notes": note_str
    })

# Add ingredients that were only in notes (unparsed)
for ingredient, note_list in notes.items():
    if ingredient not in shopping_list:
        rows.append({
            "Ingredient": ingredient,
            "Total Quantity": "",
            "Notes": ", ".join(note_list)
        })

df = pd.DataFrame(rows).sort_values("Ingredient")
display(df)


# --- Save to Excel ---
output_file = "C:\\Users\\calvi\\OneDrive\\Documents\\shopping_list.csv"
df.to_csv(output_file, index=False)

print(f"Shopping list saved to '{output_file}'")


,Ingredient,Total Quantity,Notes
6,Almond Butter,0.5 cups,
17,Bean Sprouts,0.25 lbs,
8,Cashews,0.5 cups,
15,Chinese Broccoli,1 pcs,
12,Dark Chocolate,0.5 cups,
9,Dates,5 pcs,
14,Ginger,1 tbsp,
5,Honey,1 cups,
3,Kaffir Lime Leaves,3 pcs,
4,Nectarines,2 pcs,


Shopping list saved to 'C:\Users\calvi\OneDrive\Documents\shopping_list.csv'
